## To translate or not to translate

Experimentally determining whether translating the ligand coordinates wrt to protein makes a difference

### Generate random point clouds in 3-D

In [1]:
import numpy as np
from dataset import get_mol2_coordinates_by_element, get_pdb_coordinates_by_element
from gtda.homology import VietorisRipsPersistence
import argparse
import pickle
from pathlib import Path

a = np.random.rand(200, 3)
b = np.random.rand(100, 3)

In [2]:
def compute_pairwise_opposition_distance(a, b):
    output = np.ones((len(a) + len(b), len(a) + len(b))) * np.Inf
    for idxa, ra in enumerate(a):
        for idxb, rb in enumerate(b):
            dist = np.linalg.norm((ra - rb))
            output[idxa, len(a) + idxb] = dist
            output[len(a) + idxb, idxa] = dist

    return output

dists = compute_pairwise_opposition_distance(a, b)

In [3]:
num_oppositions = 0

def opposition_homology(protein_coords, ligand_coords):
    global num_oppositions
    num_oppositions = 0
    if protein_coords is None or ligand_coords is None:
        return None

    # Track connected components, loops, and voids
    homology_dimensions = [0, 1, 2]

    # Collapse edges to speed up H2 persistence calculation!
    persistence = VietorisRipsPersistence(
        metric='precomputed',
        homology_dimensions=homology_dimensions,
        collapse_edges=True,
        max_edge_length=200
    )

    distances = compute_pairwise_opposition_distance(protein_coords, ligand_coords)
    diagrams_basic = persistence.fit_transform(distances[None, :, :])

    return diagrams_basic

def bin_persistence_diagram(diagram, bins=np.arange(0, 50, 0.25), types_of_homologies=3):
    output = np.zeros((len(bins) + 1, types_of_homologies, 3), dtype=np.int32)

    if diagram is None:
        return output

    # Bin persistence diagram
    # Binned persistence diagram be in the shape of (len(bins) + 1, types_of_homologies, 3)
    #   where the first channel corresponds to each entry,
    #   second channel is corresponds to type of homology (Betti-0, Betti-1, ...)
    #   third channel corresponds to (birth, persist, death)

    diagram = diagram[0]
    # Now diagram should be in shape (n, 3) where each row is
    #   (birth, death, type_of_homology)

    assert len(np.unique(diagram[:, 2])) == types_of_homologies

    for entry in diagram:
        homology_type = int(entry[2])
        # mark the beginning and end bins
        begin_bin = np.digitize(entry[0], bins)
        output[begin_bin, homology_type, 0] += 1
        end_bin = np.digitize(entry[1], bins)
        output[end_bin, homology_type, 2] += 1

        # mark the middle bins if there are any
        if not begin_bin == end_bin:
            output[np.arange(begin_bin + 1, end_bin), homology_type, 1] += 1

    return output

def run(protein_coords, ligand_coords):
    diagram = opposition_homology(protein_coords, ligand_coords)
    binned_diagram = bin_persistence_diagram(diagram)
    return binned_diagram


In [4]:
o1 = run(a, b)
o2 = run(a+3, b)

In [5]:
o1.shape

(201, 3, 3)

In [6]:
np.sum(o1[:, 0, :] - o2[:, 0, :])

0

## Conclusion: Empirically, translation does make a difference in Betti-1 and Betti-2, but not in Betti-0.